### Instalando las librerias necesarias

Para la correcta ejecucion del proyecto primero se deberan de instalar las librerias usadas para la implementacion del mismo.

In [120]:
!pip install anytree pandas
!pip install pytholog
!apt install swi-prolog
!pip install pykeen
!pip install -U scikit-fuzzy


The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.



### Implementacion del agente

In [121]:
import numpy as np
import pandas as pd
from anytree import NodeMixin, RenderTree
import pytholog as pl 
import math
import itertools
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [122]:
"""
Este metodo convierte cualquier lista de tareas sin orden en un arbol, para que el algoritmo de busqueda pueda luego seleccionar el mejor camino dentro del arbol. 
"""

# leer el archivo csv original
df = pd.read_csv("tasksUnordered.csv")

# ordenar las tareas por su tiempo de inicio
df = df.sort_values(by=["TimeStart"])

# crear un diccionario para almacenar los padres de cada tarea
parents = {}

# recorrer cada tarea
for i, row in df.iterrows():
    # encontrar el tiempo de finalización de la tarea actual
    current_end_time = row["TimeStart"] + row["AvgTime"]
    # inicializar la distancia más cercana como infinito
    closest_distance = float("inf")
    # inicializar el padre más cercano como None
    closest_parent = None
    # buscar el padre más cercano en las tareas anteriores
    for j in range(i):
        # encontrar el tiempo de finalización de la tarea anterior
        previous_end_time = df.iloc[j]["TimeStart"] + df.iloc[j]["AvgTime"]
        # si la tarea anterior termina antes de que comience la tarea actual
        # y la distancia entre la tarea anterior y la actual es menor que la distancia más cercana actual
        # actualizar el padre más cercano y la distancia más cercana
        if previous_end_time <= row["TimeStart"] and (row["TimeStart"] - previous_end_time) < closest_distance:
            closest_distance = row["TimeStart"] - previous_end_time
            closest_parent = int(df.iloc[j]["TaskID"])
    # si se encontró un padre cercano, agregarlo al diccionario de padres
    if closest_parent is not None:
        parents[int(row["TaskID"])] = int(closest_parent)

# agregar la columna Parent al dataframe
df["Parent"] = df["TaskID"].map(parents)

# guardar el dataframe en un nuevo archivo csv
df.to_csv("tasksOrdered.csv", index=False)


In [123]:
"""
Código que calcula la urgencia de cada tarea usando Redes de Creencias.

Definir las funciones de creencia para cada factor:
AvgTime: podemos definir una función de creencia triangular donde los valores más bajos tengan menos prioridad y los valores más altos tengan más prioridad.
"""

def creencia_avg_time(avg_time):
    if avg_time < 10:
        return 0.2
    elif avg_time < 20:
        return (avg_time - 10) / 10
    elif avg_time < 30:
        return 1 - (avg_time - 20) / 10
    else:
        return 0.2
"""
Deadline: podemos definir una función de creencia inversa, donde los valores más bajos tengan más prioridad y los valores más altos tengan menos prioridad.
"""
def creencia_deadline(deadline):
    if deadline < 1:
        return 1
    elif deadline < 5:
        return 1 - (deadline - 1) / 4
    elif deadline < 10:
        return 0.5 - (deadline - 5) / 10
    else:
        return 0
"""
IsHobby: podemos definir una función de creencia que baje un poco la prioridad si la tarea es un hobby.
"""
def creencia_is_hobby(is_hobby):
    if is_hobby:
        return 0.8
    else:
        return 1
"""
Construir la red de creencias:
Podemos construir una red de creencias simple que combine estas tres funciones de creencia usando una regla del producto:
"""
def calcular_urgencia(avg_time, deadline, is_hobby):
    creencia_avg_time_res = creencia_avg_time(avg_time)
    creencia_deadline_res = creencia_deadline(deadline)
    creencia_is_hobby_res = creencia_is_hobby(is_hobby)
    return creencia_avg_time_res * creencia_deadline_res * creencia_is_hobby_res

"""
Usar la función para calcular la urgencia de cada tarea:
Podemos iterar sobre las filas del CSV y calcular la urgencia para cada tarea, luego podemos guardar el valor de la urgencia en una nueva columna del CSV:
"""

# Definir función para escalar urgencia
def escala_urgencia(valor):
    urgencia_min = 1
    urgencia_max = 10
    urgencia = urgencia_min + ((valor - 0) / (1 - 0)) * (urgencia_max - urgencia_min)
    return round(urgencia, 2)


import pandas as pd

# Leer el CSV
df = pd.read_csv("tareas.csv")

# Calcular la urgencia para cada tarea y escalar el valor entre 1 y 10 con máximo 2 espacios decimales
df["Urgency"] = df.apply(lambda row: escala_urgencia(calcular_urgencia(row["AvgTime"], row["Deadline"], row["IsHobby"])), axis=1)

# Guardar el CSV con la nueva columna
df.to_csv("tareas_con_urgencia.csv", index=False)

In [124]:
class Task:
    def __init__(self, id, taskType, name, avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves):
        self.id=id
        self.taskType=taskType
        self.name=name
        self.avgTime=avgTime
        self.timeStart=timeStart
        self.deadline=deadline
        self.location=location
        self.urgency=urgency
        self.prioridad=""
        self.isHobby=isHobby
        self.recurrency=recurrency
        self.bestOn=bestOn
        self.involves=involves
        self.onPath=False
    
    def __str__(self) -> str:
        """
        To String del objeto Task. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Task): Instancia de la clase Task.
        
        Returns:
            str: Representacion como texto del objeto Task.
        """
        return f"id: {self.id}, taskType: {self.taskType}, name: {self.name}, avgTime: {self.avgTime}, timeStart:{self.timeStart},deadline:{self.deadline},location:{self.location} urgency: {self.urgency}, prioridad: {self.prioridad}, isHobby: {self.isHobby}, recurrency:{self.recurrency}, bestOn:{self.bestOn} "


    """
    taskType -> 
      1- Alimentación​
      2- Aseo​
      3- Actividad de Mascota​
      4- Estudio​
      5- Salud Física​
      6- Plantas​
      7- Gustos​
      8- Actividad Social​
      9- Actividad de Autocuidado y Orden
    avgTime -> Tiempo promedio (minutos)
    timeStart -> Hora en que suele comenzar esta actividad
    deadline -> Dias para entregarla (max)
    urgency -> nivel de prioridad
    location -> lugar fisico
    isHobby -> Se sabe que sea un hobby
    recurrency -> Cuantos dias de la semana suele repetirse
    bestOn-> Mejor en mañana o en la tarde
    involves -> Sujetos terceros.
    """


class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self,id, taskType, name,  avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves, parent=None, children=None):
        super().__init__(id, taskType, name, avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves)
        self.prioridad=""
        self.parent = parent
        self.onPath=False
        if children:
            self.children = children

In [125]:
class Choice():
    def __init__(self, move, urgency):
        self.move = move
        self.urgency = urgency



    def __str__(self):
        """
        To String del objeto Choice. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Choice): Instancia de la clase Choice.
        
        Returns:
            str: Representacion como texto del objeto Choice.
        """
        return self.move + ": " + str(self.urgency)
    

In [126]:
#definamos la estructura más básica de nuestro agente
#la cual podrá ir creciendo según se definan nuevas características


class ScheduleAgent:
  def __init__(self, name:str, age:int, gender:str, taskList:pd.DataFrame, comments):
    self.name = name
    self.age = age
    self.gender = gender
    self.taskList = taskList
    self.notification = comments
    self.comments = comments
    self.rootNode = self.buildTree() #creo el arbol al inicializar el objeto
    
    
  
  def getNodesList(self) -> list:
    """
    Crear una lista de nodos del arbol de tareas de acuerdo a las
    tareas almacenadas en un archivo CSV.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        list: Lista de nodos del arbol de tareas, cada nodo representa una tarea.
    """

# TaskID,TaskType,Name,AvgTime,TimeStart,Deadline,Urgency,location,isHobby,recurrency,bestOn,involves,Parent,Children

    nodeList = []
    for i in range(0, len(self.taskList["Name"])):
        nodeList.append(TaskClass(
            id=self.taskList["TaskID"][i],
            taskType = self.taskList["TaskType"][i],
            name = self.taskList["Name"][i],
            avgTime = self.taskList["AvgTime"][i],
            timeStart = self.taskList["TimeStart"][i],
            deadline = self.taskList["Deadline"][i],
            urgency = self.taskList["Urgency"][i],
            location = self.taskList["Location"][i],
            isHobby = self.taskList["IsHobby"][i],
            recurrency = self.taskList["Recurrency"][i],
            bestOn = self.taskList["BestOn"][i],
            involves = self.taskList["Involves"][i],
        ))
    return nodeList
  
  
  
  def setParentNodes(self, nodeList:list) -> TaskClass:
    """
    Colocar a cada uno de los nodos del arbol de tareas su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        nodeList (list): Lista de nodos del arbol de tareas, cada nodo representa una tarea.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    for i in range(1, len(nodeList)):
        node = nodeList[i]
        node.parent = nodeList[(int(self.taskList["Parent"][i]))-1]
    return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo

  
  
  def buildTree(self) -> TaskClass: 
    """
    Armar el arbol de tareas.
    En primera instancia crea una lista de nodos de tareas para 
    despues asignarles su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    nodeList = self.getNodesList()
    rootNode = self.setParentNodes(nodeList)
    return rootNode #retorno root del arbol ya que desde root lo puedo recorrer todo
    
    
    
  def printTaskTree(self) -> None:
    """
    Imprimir por completo el arbol de tareas. 
    
    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
    """
    for pre, fill, node in RenderTree(self.rootNode):
        print("%s%s" % (pre, node.name))   
  
  
  
  def miniMax(self, rootNode:Task, isMax:bool) -> Choice:
    """
    Algoritmo de busqueda miniMax para determinar la mayor cantidad de tareas de la mayor urgencia posible
    a completar en el minimo tiempo posible.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        rootNode (Task): Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
        isMax (bool): ?

    Returns:
        Choice: Eleccion de rama y nodo del algortimo miniMax
    """
    children = rootNode.children
    try:
        l_choice = self.miniMax(children[0], not isMax)
        r_choice = self.miniMax(children[1], not isMax)

        if (isMax):
            if (l_choice.urgency > r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency)
        else:
            if (l_choice.urgency < r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency) 
    except IndexError:
        return Choice("end", self.rootNode.urgency)

In [127]:
horario = {}

def agregar_tarea(node):
    horario[node.name] = node

def ver_horario():
    print(horario)

In [128]:
#inicializando el agente para el usuario Ricardo
myAgent = ScheduleAgent(name="Ricardo",
                        age=36,
                        gender="Male",
                        taskList=pd.read_csv('agentTaskList.csv'),
                        comments = "")

isMax = True
currentNode = myAgent.rootNode
agregar_tarea(currentNode)

print("Comenzamos nuestro recorrido en el nodo: "+str(currentNode.name)+", con una urgencia de: "+str(currentNode.urgency))
currentNode.onPath=True

while (True):
    # run minimax on current node
    agentChoice = myAgent.miniMax(currentNode, isMax) 
    
    # make choice based on minimax search
    if (agentChoice.move == "left"):
        print ("Nos movemos a la izquierda con una urgencia de: " + str(currentNode.children[0].urgency)+ ", a la tarea: "+str(currentNode.children[0].name))
        currentNode.children[0].onPath=True
        currentNode = currentNode.children[0]
        agregar_tarea(currentNode)
    elif (agentChoice.move == "right"):
        print ("Nos movemos a la derecha con una urgencia de: " + str(currentNode.children[1].urgency)+ ", a la tarea: "+str(currentNode.children[1].name))
        currentNode.children[1].onPath=True
        currentNode = currentNode.children[1]
        agregar_tarea(currentNode)
    elif (agentChoice.move == "end"):
        print ("Hemos llegado al final del mejor recorrido")
        break

Comenzamos nuestro recorrido en el nodo: desayunar, con una urgencia de: 5
Nos movemos a la derecha con una urgencia de: 5, a la tarea: sacar al perro
Nos movemos a la derecha con una urgencia de: 7, a la tarea: regar plantas
Nos movemos a la derecha con una urgencia de: 6, a la tarea: jugar con el perro
Hemos llegado al final del mejor recorrido


In [129]:
ver_horario()

{'desayunar': <__main__.TaskClass object at 0x242fafb20>, 'sacar al perro': <__main__.TaskClass object at 0x242fac5e0>, 'regar plantas': <__main__.TaskClass object at 0x242fac8b0>, 'jugar con el perro': <__main__.TaskClass object at 0x242fafc40>}


In [130]:
# PARA ORGANIZAR LAS TAREAS SEGUN SUS ATRIBUTOS.

dictTasks = {}
dictHobbies = {}
dictHabits = {}

In [131]:
# AÑADIR LAS TAREAS DE LA ESTRUCTURA ANYTREE AL ARCHIVO DE LOGICA DE PROLOG

# reinicia entre corridas el txt, aqui se ponen la reglas basicas de la logica del agente
with open("logica_agente.txt", "w") as archivo:
    archivo.write("sameLocation(X,Y) :- inLocation(X,Z), inLocation(Y,Z) -> sameLocation(X,Y). " + '\n')
archivo.close()

with open('logica_agente.txt', mode='a') as archivo:
    archivo.write("\nisAgent("+myAgent.name+")")
archivo.close()


# METODO PARA AÑADIR LOS TASKS
def addAllTasks(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                    dictTasks[node.name] = node
                    archivo.write("isTask("+node.name + ")" + '\n')
                    archivo.write("inLocation("+node.name + "," + node.location +")" + '\n')
                    archivo.write("hasDeadline("+node.name + "," + str(node.deadline) +")" + '\n')
                    archivo.write("hasUrgency("+node.name + "," + str(node.urgency) +")" + '\n')
                    archivo.write("isBestOn("+node.name + "," + node.bestOn +")" + '\n')
                    if node.onPath == True:
                        archivo.write("inTodaysSchedule("+node.name + ")" + '\n')
                    if node.involves != "" and node.involves != np.nan:
                        archivo.write("involvesWho("+node.name + "," + str(node.involves) +")" + '\n')
                    
    archivo.close()

# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HOBBIES
def addHobbies(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                if node.isHobby == True: 
                    dictHobbies[node.name] = node
                    archivo.write("isHobby("+node.name + ")" + '\n')
    archivo.close()


# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HABITOS
def addHabits(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                if node.recurrency >= 5: 
                    dictHabits[node.name] = node
                    archivo.write("isHabit("+node.name + ")" + '\n')
    archivo.close()
                
               

addAllTasks(myAgent.rootNode)
addHobbies(myAgent.rootNode)
addHabits(myAgent.rootNode)



# Base de conocimiento
KB2 = pl.KnowledgeBase("KB2")
KB2.from_file("logica_agente.txt")



facts and rules have been added to KB2.db


In [132]:
print("Tasks: " + str(dictTasks))
print(dictTasks['bañarse'])
print(dictTasks['bañarse'].location)
print("")
print("Hobbies: " + str(dictHobbies))
print("")
print("Habits: " + str(dictHabits))

Tasks: {'bañarse': <__main__.TaskClass object at 0x242facf70>, 'hacer la tarea': <__main__.TaskClass object at 0x242fac5b0>, 'almorzar': <__main__.TaskClass object at 0x242fac940>, 'hacer ejercicio': <__main__.TaskClass object at 0x242fac8e0>, 'estudiar para el parcial': <__main__.TaskClass object at 0x242faf7c0>, 'escuchar musica': <__main__.TaskClass object at 0x242fac910>, 'leer documentacion': <__main__.TaskClass object at 0x242facc10>, 'sacar al perro': <__main__.TaskClass object at 0x242fac5e0>, 'hacer estiramientos': <__main__.TaskClass object at 0x242faf7f0>, 'ver netflix': <__main__.TaskClass object at 0x242facbb0>, 'reunirse con amigos': <__main__.TaskClass object at 0x242fafc10>, 'regar plantas': <__main__.TaskClass object at 0x242fac8b0>, 'ordenar el cuarto': <__main__.TaskClass object at 0x242fafcd0>, 'jugar con el perro': <__main__.TaskClass object at 0x242fafc40>}
id: 2, taskType: 2, name: bañarse, avgTime: 45, timeStart:8.45,deadline:1,location:casa urgency: 2, priorida

In [133]:
# Ejemplos de Queries

# Queries sobre nuestros datos

print("Preguntamos sobre una tarea que no esta en el dataframe")
print(KB2.query(pl.Expr("isTask(desayunar)")))

print("Preguntamos datos sobre una tarea que sí esta en el dataframe")
print(KB2.query(pl.Expr("isTask(almorzar)")))

print("Preguntamos por clasificaciones")
print(KB2.query(pl.Expr("isHobby(almorzar)")))
print(KB2.query(pl.Expr("isHabit(almorzar)")))

print("Preguntamos datos sobre relaciones entre tareas")
print(KB2.query(pl.Expr("sameLocation(almorzar,regar plantas)")))

print("Distinguimos que tareas son las que se agregaron al horario de hoy ")
print(KB2.query(pl.Expr("inTodaysSchedule(almorzar)"))) 
print(KB2.query(pl.Expr("inTodaysSchedule(regar plantas)")))

Preguntamos sobre una tarea que no esta en el dataframe
['No']
Preguntamos datos sobre una tarea que sí esta en el dataframe
['Yes']
Preguntamos por clasificaciones
['No']
['Yes']
Preguntamos datos sobre relaciones entre tareas
['No']
Distinguimos que tareas son las que se agregaron al horario de hoy 
['No']
['Yes']


In [134]:
# Definir variables de entrada y salida
urgencia = ctrl.Antecedent(np.arange(1, 11, 1), 'urgencia')
prioridad = ctrl.Consequent(np.arange(0, 11, 1), 'prioridad')

# Definir los conjuntos difusos y las funciones de pertenencia correspondientes
urgencia['muy_baja'] = fuzz.trimf(urgencia.universe, [1, 1, 3])
urgencia['baja'] = fuzz.trimf(urgencia.universe, [1, 3, 5])
urgencia['media'] = fuzz.trimf(urgencia.universe, [3, 5, 7])
urgencia['alta'] = fuzz.trimf(urgencia.universe, [5, 7, 9])
urgencia['muy_alta'] = fuzz.trimf(urgencia.universe, [7, 10, 10])

prioridad['baja'] = fuzz.trimf(prioridad.universe, [0, 0, 5])
prioridad['media'] = fuzz.trimf(prioridad.universe, [0, 5, 10])
prioridad['alta'] = fuzz.trimf(prioridad.universe, [5, 10, 10])

# Definir las reglas difusas
regla1 = ctrl.Rule(urgencia['muy_baja'], prioridad['baja'])
regla2 = ctrl.Rule(urgencia['baja'], prioridad['baja'])
regla3 = ctrl.Rule(urgencia['media'], prioridad['media'])
regla4 = ctrl.Rule(urgencia['alta'], prioridad['alta'])
regla5 = ctrl.Rule(urgencia['muy_alta'], prioridad['alta'])

# Definir el sistema de control difuso
sistema = ctrl.ControlSystem([regla1, regla2, regla3, regla4, regla5])
asignar_prioridad = ctrl.ControlSystemSimulation(sistema)

def escribirPrioridad(prioridad):
    # Imprimir la prioridad asignada en forma de cadena de texto
    if prioridad_tarea <= 2.5:
        print("La prioridad de la tarea es baja")
    elif prioridad_tarea <= 7.5:
        print("La prioridad de la tarea es media")
    else:
        print("La prioridad de la tarea es alta")




for clave, valor in dictTasks.items():
    print("asigno Prioridad para: " +clave +", que tiene urgencia: "+ str(valor.urgency))

    # Asignar un valor de urgencia para la tarea de desayunar
    asignar_prioridad.input['urgencia'] = valor.urgency

    # Calcular el resultado del sistema de control difuso
    asignar_prioridad.compute()

    # Obtener la salida (prioridad)
    prioridad_tarea = asignar_prioridad.output['prioridad']
    print('La prioridad de la tarea es:', prioridad_tarea)
    valor.prioridad= prioridad_tarea
    escribirPrioridad(valor.prioridad)





asigno Prioridad para: bañarse, que tiene urgencia: 2
La prioridad de la tarea es: 1.9444444444444444
La prioridad de la tarea es baja
asigno Prioridad para: hacer la tarea, que tiene urgencia: 7
La prioridad de la tarea es: 8.333333333333334
La prioridad de la tarea es alta
asigno Prioridad para: almorzar, que tiene urgencia: 4
La prioridad de la tarea es: 4.404761904761905
La prioridad de la tarea es media
asigno Prioridad para: hacer ejercicio, que tiene urgencia: 3
La prioridad de la tarea es: 1.6666666666666665
La prioridad de la tarea es baja
asigno Prioridad para: estudiar para el parcial, que tiene urgencia: 9
La prioridad de la tarea es: 8.194444444444445
La prioridad de la tarea es alta
asigno Prioridad para: escuchar musica, que tiene urgencia: 1
La prioridad de la tarea es: 1.6666666666666665
La prioridad de la tarea es baja
asigno Prioridad para: leer documentacion, que tiene urgencia: 5
La prioridad de la tarea es: 5.000000000000001
La prioridad de la tarea es media
asign

In [135]:
# Crear un diccionario vacío para almacenar las tareas que no están en horario
nuevas_tareas = {}

# Iterar sobre las claves en dictTasks
for tarea in dictTasks.keys():
    
    # Comprobar si la tarea no está en horario
    if tarea not in horario:
        
        # Si no está en horario, agregarla al nuevo diccionario
        nuevas_tareas[tarea] = dictTasks[tarea]



print("En caso de no poder realizar alguna de las tareas en tu horario te recomendamos tomar alguna de alta prioridad,. \n a continuacion te las listamos en el orden recomendado: \n")

# Ordenar las tareas por prioridad (de mayor a menor)
tareas_ordenadas = sorted(nuevas_tareas.values(), key=lambda x: x.prioridad, reverse=True)

# Recorrer las tareas ordenadas e imprimir su prioridad
for tarea in tareas_ordenadas:
    print("- " +tarea.name + ", con una prioridad de: " + str(round(tarea.prioridad, 2)))

En caso de no poder realizar alguna de las tareas en tu horario te recomendamos tomar alguna de alta prioridad,. 
 a continuacion te las listamos en el orden recomendado: 

- hacer la tarea, con una prioridad de: 8.33
- estudiar para el parcial, con una prioridad de: 8.19
- leer documentacion, con una prioridad de: 5.0
- almorzar, con una prioridad de: 4.4
- reunirse con amigos, con una prioridad de: 4.4
- ordenar el cuarto, con una prioridad de: 4.4
- bañarse, con una prioridad de: 1.94
- hacer estiramientos, con una prioridad de: 1.94
- hacer ejercicio, con una prioridad de: 1.67
- escuchar musica, con una prioridad de: 1.67
- ver netflix, con una prioridad de: 1.67
